In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
pieces = pd.read_csv("geometry.csv",index_col=0)


In [ ]:
data = pieces.copy()

In [ ]:
# scale data to be between 1 to 10
data = data.apply(lambda x: (x - x.min()) / (x.max() - x.min()) * 9 +1)

In [ ]:
# initialize random centroids
def random_centroids(data, k):
    centroids = []
    for i in range(k):
        centroids.append(data.apply(lambda x: float(x.sample())))
    return pd.concat(centroids, axis=1)



In [ ]:
centroid_count = 10
max_iterations = 100
centroids = random_centroids(data, centroid_count)

In [ ]:
distances = centroids.apply(lambda x: np.sqrt(((data-x)** 2).sum(axis=1)))

In [ ]:
def get_labels(data, centroids):
    distances = centroids.apply(lambda x: np.sqrt(((data - x) ** 2).sum(axis=1)))
    return distances.idxmin(axis=1)

In [ ]:
labels = get_labels(data, centroids)

In [ ]:
labels.value_counts()

In [ ]:
data.groupby(labels).apply(lambda x: np.exp(np.log(x).mean())).T

In [ ]:
def new_centroids(data, labels, k):
    centroids = data.groupby(labels).apply(lambda x: np.exp(np.log(x).mean())).T
    return centroids

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
def plot_clusters(data, labels, centroids, iteration):
    pca = PCA(n_components=2)
    data_2d = pca.fit_transform(data)
    centroids_2d = pca.transform(centroids.T)
    clear_output(wait=True)
    plt.title(f'Iteration {iteration}')
    plt.scatter(x=data_2d[:,0], y=data_2d[:,1], c=labels)
    plt.scatter(x=centroids_2d[:,0], y=centroids_2d[:,1])
    plt.show()

In [ ]:



centroids = random_centroids(data, centroid_count)
old_centroids = pd.DataFrame()
iteration = 1

while iteration < max_iterations and not centroids.equals(old_centroids):
    old_centroids = centroids
    
    labels = get_labels(data, centroids)
    centroids = new_centroids(data, labels, centroid_count)
    # plot_clusters(data, labels, centroids, iteration)
    iteration += 1

In [ ]:
centroids

In [ ]:
print(labels)

In [ ]:
# print all pices with their cluster label is 1
pieces[labels == 0]

In [ ]:
pieces[labels == 1
       ]

In [ ]:
pieces[labels == 2]

In [ ]:
pieces[labels == 4]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(centroid_count)
kmeans.fit(data)

In [ ]:
pd.DataFrame(kmeans.cluster_centers_).T

In [7]:
import cv2
from os.path import join
# Load the image
img = cv2.imread(join('threshold', 'Page_0001_1_4'+".png"))

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Find the corners using the Harris corner detector
dst = cv2.cornerHarris(gray, 1, 9, 0.04)

# Normalize the output
dst_norm = cv2.normalize(dst, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32FC1)

# Threshold to obtain the corners
threshold = 0.01
corners = cv2.threshold(dst_norm, threshold*dst_norm.max(), 255, cv2.THRESH_BINARY)[1]

# Draw the corners on the original image
img[corners > 0] = [0, 0, 0]

# Display the result
cv2.imshow('Corners', img)
cv2.waitKey(0)
cv2.destroyAllWindows()